## Autoregressor Approach Verification

This notebook verifies the approach that was taken to forecast the HDI and the WDI data. 

#### HDI Data Forecast
In general, HDI data was forcasted for the years 2020, 2021 and 2022 in the main notebook. As a test and to verify that the chosen approach delivers accurate results, all entries belonging to the year 2019 were chosen as a test set. The actual 2019 values were then compared to the forecasted 2019 values and some error measures were computed. 

#### WDI Data Forecast
In general, WDI data was forcasted for the years 2022 in the main notebook. As a test and to verify that the chosen approach delivers accurate results, all entries belonging to the year 2021 were chosen as a test set. The actual 2021 values were then compared to the forecasted 2021 values and some error measures were computed. 

In [1]:
#import necessary packages
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

### 1) WDI Data Forecast

In [2]:
#importing the WDI data from the other notebook
df_wbi_final = pd.read_excel("autoreg_test_wbi.xlsx")

In [3]:
#taking a look at the dataframe
df_wbi_final.head(2)

,year,Access to clean fuels and technologies for cooking (% of population),Access to electricity (% of population),Consumer price index (2010 = 100),"Employment to population ratio, 15+, total (%) (modeled ILO estimate)",GDP (constant 2015 US$),GDP per capita (constant 2015 US$),Oil rents (% of GDP),Refugee population by country or territory of asylum,Rural population (% of total population),"Unemployment, female (% of female labor force) (modeled ILO estimate)","Unemployment, male (% of male labor force) (modeled ILO estimate)","Unemployment, total (% of total labor force) (modeled ILO estimate)",Country Code
0,1982,1.152381,14.680897,69.858202,67.273226,4.941037e+08,609.702019,0.0,8800.53125,79.987,7.038645,5.415355,6.180968,GNB
1,1982,2.709524,39.431974,8.020179,52.774806,4.993221e+08,736.342689,0.0,7953.43750,69.887,12.432774,7.109903,9.349548,GMB


In [4]:
#split off the 2021 data into the test set and the remaining data as the train set
df_wbi_final_test = df_wbi_final[df_wbi_final['year'] == 2021]
df_wbi_final_train = df_wbi_final[df_wbi_final['year'] != 2021]

len(df_wbi_final_test), len(df_wbi_final_train)

(14, 546)

In [36]:
#use the same function as in the main notebook to predict the values for 2021
def create_predictions(data, columns, States, year_list):
    df_pred = pd.DataFrame()
    df_new = pd.DataFrame()
    df = pd.DataFrame()
    steps = len(year_list)
     
    for j in States: 
        data_filtered = data[data['Country Code'] == j]
        data_filtered.drop(columns = ['Country Code'], inplace = True)
        data_train = data_filtered
        
        for i in columns: 
            forecaster = ForecasterAutoreg(regressor = RandomForestRegressor(random_state=123), lags = 15)
            
            forecaster.fit(y=data_train[i])
            
            predictions = forecaster.predict(steps = steps)
            predictions.to_frame()
            df_pred[i] = predictions
            df_pred['Country Code'] = j
            df_pred['year'] = year_list
        df_new = df_new.append(df_pred)
    df = df.append(df_new)
   
    return df

In [37]:
#call the prediction function to predict the values
country_codes = set(df_wbi_final["Country Code"])
year_list3 = [2021]
predictors = ['Access to clean fuels and technologies for cooking (% of population)',
 'Access to electricity (% of population)',
 'Consumer price index (2010 = 100)',
 'Employment to population ratio, 15+, total (%) (modeled ILO estimate)',
 'GDP (constant 2015 US$)',
 'GDP per capita (constant 2015 US$)',
 'Oil rents (% of GDP)',
 'Refugee population by country or territory of asylum',
 'Rural population (% of total population)',
 'Unemployment, female (% of female labor force) (modeled ILO estimate)',
 'Unemployment, male (% of male labor force) (modeled ILO estimate)',
 'Unemployment, total (% of total labor force) (modeled ILO estimate)']

df = create_predictions(df_wbi_final_train, predictors, country_codes, year_list3)

C:\Users\Shadow\AppData\Local\Temp\ipykernel_20716\2963614226.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_filtered.drop(columns = ['Country Code'], inplace = True)
C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
C:\Us

C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index i

C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index i

C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index i

C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index i

C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index i

C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index i

C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
C:\Users\Shadow\AppData\Local\Temp\ipykernel_20716\2963614226.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_new = df_new.append(df_pred)
C:\Users\Shadow\AppData\Local\Temp\ipykernel_20716\2963614226.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of

C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index i

C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
C:\Users\Shadow\AppData\Local\Temp\ipykernel_20716\2963614226.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_new = df_new.append(df_pred)
C:\Users\Shadow\AppData\Local\Temp\ipykernel_20716\2963614226.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_new)


In [38]:
#group the predicted values and the original test set so that both dataframes have the same structure
df_wbi_final_grouped = df_wbi_final_test.groupby(["Country Code", "year"]).sum()
df = df.groupby(["Country Code", "year"]).sum()

In [46]:
#compute the differences of both dataframes and calculate the mean error as well as the mean average percentage error
df_differences = df.subtract(df_wbi_final_grouped, axis = 1)

In [47]:
df_differences = df_differences.abs()
df_differences = df_differences.mean()

In [48]:
df_means = df.mean()

df_mpe = df_differences/df_means

df_mpe.mean()

0.13298568867377894

The average percentage error for the predictions across all predictors for the WDI dataset is roughly 13% which validates the chosen approach.  

### 2) HDI Data Forecast

In [49]:
#importing the HDI data from the other notebook
df_hdi = pd.read_excel("autoreg_hdi.xlsx")

In [50]:
#defining necessary lists
adm1 = set(df_hdi["ActionGeo_ADM1Code"])
year_list1 = [201901, 201902, 201903, 201904, 201905, 201906, 201907, 201908, 201909, 201910, 201911, 201912]
numeric_columns = ['gender development index (GDI)', 'human development index (HDI)',
       'health index', 'income index', 'educational index',
       'expected years of schooling', 'mean years of scholing',
       'life expectancy', "GNI per capita (000'000)", "population (000'000)"]

In [51]:
#taking a look at the HDI data
df_hdi.head(2)

,MonthYear_2,MonthYear,Year,ActionGeo_CountryCode,ActionGeo_ADM1Code,number of conflicts,level,region,gender development index (GDI),human development index (HDI),health index,income index,educational index,expected years of schooling,mean years of scholing,life expectancy,GNI per capita (000'000),population (000'000)
0,198201,198201,1982,GH,GH00,14,NaN,NaN,0.890083,0.529633,0.607433,0.525700,0.4662,9.230000,6.296667,59.4900,8.085167,21989.604833
1,198201,198201,1982,MR,MR08,0,NaN,NaN,0.888000,0.575933,0.696267,0.730633,0.3793,7.629333,5.021800,65.2566,9.442367,110.328167


In [52]:
#split off the 2019 data into the test set and the remaining data as the train set
df_hdi_test = df_hdi[df_hdi['Year'] == 2019]
df_hdi_train = df_hdi[df_hdi['Year'] != 2019]

len(df_hdi_test), len(df_hdi_train)

(2460, 91020)

In [53]:
#use the same function as in the main notebook to predict the values for 2019
def create_predictions(data, predictors, States, year_list):
    df_pred = pd.DataFrame()
    df_new = pd.DataFrame()
    df = pd.DataFrame()
    steps = len(year_list)
     
    for j in States: 
        data_filtered = data[data['ActionGeo_ADM1Code'] == j]
        data_filtered.drop(columns = [#"Year", "ActionGeo_CountryCode",
            "ActionGeo_ADM1Code",
                          #"number of conflicts", #"iso_code", "country_x", "year", 
                                    #  "level", "region",
                          #"adm1", "country_y"
                                     ], inplace = True)
        data_train = data_filtered
        
        for i in predictors: 
            forecaster = ForecasterAutoreg(regressor = RandomForestRegressor(random_state=123), lags = 62)
            
            forecaster.fit(y=data_train[i])
            
            predictions = forecaster.predict(steps = steps)
            predictions.to_frame()
            df_pred[i] = predictions
            df_pred['ActionGeo_ADM1Code'] = j
            df_pred['MonthYear'] = year_list
        df_new = df_new.append(df_pred)
    df = df.append(df_new)
   
    return df

In [54]:
#call the prediction function and let it run for the coming year
df = create_predictions(df_hdi_train, numeric_columns, adm1, year_list1)

C:\Users\Shadow\AppData\Local\Temp\ipykernel_20716\2581992317.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_filtered.drop(columns = [#"Year", "ActionGeo_CountryCode",
C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
C:\U

C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index i

C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index i

C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index i

C:\Users\Shadow\AppData\Local\Temp\ipykernel_20716\2581992317.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_filtered.drop(columns = [#"Year", "ActionGeo_CountryCode",
C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
C:\U

...

C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index i

C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index i

C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index i

C:\Users\Shadow\AppData\Local\Temp\ipykernel_20716\2581992317.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_filtered.drop(columns = [#"Year", "ActionGeo_CountryCode",
C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
C:\U

C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
C:\Users\Shadow\AppData\Roaming\Python\Python39\site-packages\skforecast\utils\utils.py:256: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index i

In [55]:
#drop some columns from the test dataset to ensure the same structure for both datasets
df_hdi_test.drop(columns = ["MonthYear_2", "number of conflicts", "level", "region"], inplace = True)
df_hdi_test.head(2)

C:\Users\Shadow\AppData\Local\Temp\ipykernel_20716\2103243698.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_hdi_test.drop(columns = ["MonthYear_2", "number of conflicts", "level", "region"], inplace = True)


,MonthYear,Year,ActionGeo_CountryCode,ActionGeo_ADM1Code,gender development index (GDI),human development index (HDI),health index,income index,educational index,expected years of schooling,mean years of scholing,life expectancy,GNI per capita (000'000),population (000'000)
91020,201901,2019,IV,IV92,0.810,0.539,0.582,0.593,0.454,10.000,5.300,57.800,8.531,25716.544
91021,201901,2019,BN,BN17,0.875,0.574,0.661,0.532,0.539,14.643,3.963,62.957,8.130,1814.191


In [56]:
df_hdi_test.drop(columns = [ "Year", "ActionGeo_CountryCode"], inplace = True)
df_hdi_test # "MonthYear_2", "number of conflicts", "level", "region",

C:\Users\Shadow\AppData\Local\Temp\ipykernel_20716\3020580186.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_hdi_test.drop(columns = [ "Year", "ActionGeo_CountryCode"], inplace = True)


,MonthYear,ActionGeo_ADM1Code,gender development index (GDI),human development index (HDI),health index,income index,educational index,expected years of schooling,mean years of scholing,life expectancy,GNI per capita (000'000),population (000'000)
91020,201901,IV92,0.810,0.539,0.582,0.593,0.454,10.000,5.300,57.800,8.531,25716.544
91021,201901,BN17,0.875,0.574,0.661,0.532,0.539,14.643,3.963,62.957,8.130,1814.191
91022,201901,BN02,0.863,0.639,0.653,0.584,0.684,16.333,6.908,62.459,8.469,1998.997
91023,201901,PU06,0.000,0.467,0.524,0.417,0.467,12.862,3.295,54.051,7.368,193.398
91024,201901,GV12,0.816,0.478,0.640,0.480,0.354,9.400,2.800,61.600,7.785,12771.246
...,...,...,...,...,...,...,...,...,...,...,...,...
93475,201912,UV00,0.868,0.452,0.640,0.462,0.312,9.300,1.600,61.600,7.665,20321.378
93476,201912,TO00,0.822,0.516,0.631,0.419,0.519,12.700,5.000,61.000,7.379,8082.366
93477,201912,SL04,0.882,0.452,0.534,0.425,0.407,10.200,3.700,54.700,7.419,7813.215
93478,201912,SL02,0.882,0.452,0.534,0.425,0.407,10.200,3.700,54.700,7.419,7813.215


In [57]:
#group the predicted values and the original test set so that both dataframes have the same structure
df_hdi_grouped = df_hdi_test.groupby([ "MonthYear", "ActionGeo_ADM1Code"]).sum()
df_grouped = df.groupby(["MonthYear", "ActionGeo_ADM1Code"]).sum()

In [58]:
#compute the differences of both dataframes and calculate the mean error as well as the mean average percentage error
df_differences = df_grouped.subtract(df_hdi_grouped, axis = 1)

df_differences = df_differences.abs()
df_differences = df_differences.mean()

In [59]:
df_means = df.mean()

df_mpe = df_differences/df_means

df_mpe.mean()

C:\Users\Shadow\AppData\Local\Temp\ipykernel_20716\1600898340.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_means = df.mean()


0.017874598651036265

The average percentage error for the predictions across all predictors for the HDI dataset is less than 2% which validates the chosen approach. The error for the HDI dataset is likely lower than for the WDI dataset due to being recorded on a monthly basis and therefore having many more data points. 